In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W0224 18:59:27.708060 4594804160 deprecation.py:323] From <ipython-input-1-8bf8ae5a5303>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0224 18:59:27.709332 4594804160 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0224 18:59:27.710293 4594804160 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0224 18:59:27.939557 4594804160 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import os

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, plot_model
from keras.datasets import mnist

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image

import numpy as np
np.random.seed(1)

import random
import math

Using TensorFlow backend.


In [3]:
(x_train, _), (_, _) = mnist.load_data()

x_train = x_train.reshape(-1, 28*28)/255.

In [4]:
x_dim = x_train.shape[1] # 784
z_dim = 100
h_dim = 128
batch_size = 128

x = Input(shape=(x_dim,))
D_h = Dense(h_dim)(x)
D_h = LeakyReLU(0.2)(D_h)
D_o = Dense(1, activation='sigmoid')(D_h)
D = Model(x, D_o)

z = Input(shape=(z_dim,))
G_h = Dense(h_dim)(z)
G_h = LeakyReLU(0.2)(G_h)
G_o = Dense(x_dim, activation='sigmoid')(G_h)
G = Model(z, G_o)

# del D
# D = load_model('D.h5')
# del G
# G = load_model('G.h5')

DM = Sequential()
D.trainable = True
DM.add(D)
DM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

AM = Sequential()
AM.add(G)
D.trainable = False
AM.add(D)
AM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

G.summary()
D.summary()
DM.summary()
AM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               12928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 784)               101136    
Total params: 114,064
Trainable params: 114,064
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dens

/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [5]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m,n])

if not os.path.exists('out/'):
    os.makedirs('out/')

i=0
for it in range(1000000):
    r = np.random.randint(0, x_train.shape[0], batch_size)
    x_real_batch = x_train[r]
    
    samples = sample_Z(batch_size, z_dim)
    x_fake_batch = G.predict(samples)
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    x_D = np.concatenate([x_real_batch, x_fake_batch])
    y_D = np.concatenate([real, fake])
    
    D_loss, D_acc = DM.train_on_batch(x_D, y_D)
    
    samples = sample_Z(batch_size, z_dim)
    
    G_loss, G_acc = AM.train_on_batch(samples, real)
    
    if it%1000==0:
        D.save('D.h5')
        G.save('G.h5')
        
        samples = G.predict(sample_Z(16, z_dim))
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i+=1
        plt.close(fig)
        
        print('iter:{}'.format(it))
        print('[D loss:{:.4}, acc:{:.4}] [G loss:{:.4}, acc:{:.4}]'.format(D_loss, D_acc, G_loss, G_acc))
        print()

iter:0
[D loss:0.4933, acc:0.5938] [G loss:0.6846, acc:0.4922]

iter:1000
[D loss:0.1039, acc:0.9844] [G loss:3.351, acc:0.007812]

iter:2000
[D loss:0.1122, acc:0.9766] [G loss:4.292, acc:0.05469]

iter:3000
[D loss:0.3262, acc:0.875] [G loss:4.648, acc:0.007812]

iter:4000
[D loss:0.9009, acc:0.582] [G loss:1.804, acc:0.2734]

iter:5000
[D loss:0.4827, acc:0.7656] [G loss:3.641, acc:0.1328]

iter:6000
[D loss:1.117, acc:0.4648] [G loss:1.442, acc:0.2656]

iter:7000
[D loss:0.8408, acc:0.6016] [G loss:2.164, acc:0.1562]

iter:8000
[D loss:0.6239, acc:0.7109] [G loss:2.359, acc:0.1016]

iter:9000
[D loss:0.7168, acc:0.6484] [G loss:1.765, acc:0.25]

iter:10000
[D loss:0.7662, acc:0.5781] [G loss:1.502, acc:0.25]

iter:11000
[D loss:0.9894, acc:0.5273] [G loss:1.369, acc:0.3281]

iter:12000
[D loss:0.7875, acc:0.6289] [G loss:1.63, acc:0.3359]

iter:13000
[D loss:1.07, acc:0.5039] [G loss:1.516, acc:0.4453]

iter:14000
[D loss:0.6681, acc:0.6641] [G loss:1.474, acc:0.2109]

iter:15000
[

KeyboardInterrupt: 